# Retrieval & Candidate Fusion
- Behavior-based retrieval (two-tower style embeddings)
- Content-based retrieval (precomputed item content vectors)
- Popularity / trending retrieval (simple baseline)
- Candidate fusion: **merge + quotas + dedupe**
- Evaluation: candidate **Recall@K** and coverage checks


## 0) Setup

In [1]:
import numpy as np
from numpy.random import default_rng
import math

rng = default_rng(11)

def l2_normalize(x, axis=1, eps=1e-12):
    n = np.linalg.norm(x, axis=axis, keepdims=True)
    return x / np.maximum(n, eps)

def recall_at_k(scores, pos_item, k=200):
    topk = np.argpartition(-scores, kth=k-1)[:k]
    return 1.0 if int(pos_item) in topk else 0.0


## 1) Synthetic environment: users, items, implicit feedback

In [2]:
n_users = 8000
n_items = 3000
d_true = 32

U_true = rng.normal(size=(n_users, d_true)).astype(np.float32)
V_true = rng.normal(size=(n_items, d_true)).astype(np.float32)

pop = rng.power(a=2.1, size=n_items).astype(np.float32)
pop = pop / pop.sum()

def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))

def click_prob(u, i):
    a = float(U_true[u] @ V_true[i])
    a = max(min(a, 8.0), -8.0)
    p = sigmoid(a) * 0.8 + 0.2 * float(pop[i] * n_items) / 6.0
    return float(min(max(p, 0.001), 0.999))

def sample_impressions(n=400):
    return rng.choice(n_items, size=n, replace=False, p=pop).astype(np.int32)

n_train_users = 6000
impr_per_user = 70
train_rows = []
for u in range(n_train_users):
    impr = sample_impressions(impr_per_user)
    for i in impr:
        y = 1 if rng.random() < click_prob(u, int(i)) else 0
        train_rows.append((u, int(i), y))

train = np.array(train_rows, dtype=np.int32)
print("train rows:", train.shape, "click rate:", round(train[:,2].mean(), 4))


train rows: (420000, 3) click rate: 0.4363


## 2) Behavior retriever: two-tower embeddings

In [3]:
d = 48
user_emb = (0.01 * rng.normal(size=(n_users, d))).astype(np.float32)
item_emb_behavior = (0.01 * rng.normal(size=(n_items, d))).astype(np.float32)

pos_by_user = {}
for u, i, y in train:
    if y == 1:
        pos_by_user.setdefault(int(u), []).append(int(i))

users_with_pos = np.array([u for u in range(n_train_users) if u in pos_by_user], dtype=np.int32)

counts = np.bincount(train[:,1], minlength=n_items).astype(np.float64) + 1.0
neg_dist = (counts ** 0.75); neg_dist /= neg_dist.sum()

def train_two_tower(steps=100_000, lr=0.06, neg_k=12, seed=11):
    rng2 = default_rng(seed)
    def sig(x): return 1.0 / (1.0 + np.exp(-x))
    loss = 0.0
    for t in range(steps):
        u = int(rng2.choice(users_with_pos))
        i_pos = int(rng2.choice(pos_by_user[u]))
        negs = rng2.choice(n_items, size=neg_k, replace=True, p=neg_dist).astype(np.int32)

        uvec = user_emb[u]
        posvec = item_emb_behavior[i_pos]

        s_pos = float(uvec @ posvec)
        p_pos = sig(s_pos)
        loss += -math.log(max(p_pos, 1e-10))
        g_u = (p_pos - 1.0) * posvec
        g_pos = (p_pos - 1.0) * uvec

        for j in negs:
            jvec = item_emb_behavior[int(j)]
            s_neg = float(uvec @ jvec)
            p_neg = sig(s_neg)
            loss += -math.log(max(1.0 - p_neg, 1e-10))
            grad = p_neg
            g_u += grad * jvec
            item_emb_behavior[int(j)] -= lr * (grad * uvec)

        user_emb[u] -= lr * g_u
        item_emb_behavior[i_pos] -= lr * g_pos

        if (t+1) % 25000 == 0:
            print(f"step {t+1}/{steps} avg_loss≈{loss/(t+1):.4f}")
    return loss / steps

train_two_tower()

U_beh = l2_normalize(user_emb)
I_beh = l2_normalize(item_emb_behavior)
print("behavior emb:", U_beh.shape, I_beh.shape)


step 25000/100000 avg_loss≈9.0108
step 50000/100000 avg_loss≈9.0095
step 75000/100000 avg_loss≈8.9611
step 100000/100000 avg_loss≈8.3578
behavior emb: (8000, 48) (3000, 48)


## 3) Content retriever: precomputed item content vectors

In [4]:
d_content = 64
A = rng.normal(size=(d_true, d_content)).astype(np.float32)
item_emb_content = (V_true @ A + 0.6 * rng.normal(size=(n_items, d_content)).astype(np.float32)).astype(np.float32)
item_emb_content = l2_normalize(item_emb_content)
print("content emb:", item_emb_content.shape)


content emb: (3000, 64)


## 4) Eval users and true target

In [5]:
eval_users = np.arange(6000, 7600, dtype=np.int32)

def sample_true_positive(u):
    impr = sample_impressions(600)
    probs = np.array([click_prob(int(u), int(i)) for i in impr], dtype=np.float64)
    probs /= probs.sum()
    return int(rng.choice(impr, p=probs))

pos_targets = np.array([sample_true_positive(int(u)) for u in eval_users], dtype=np.int32)
print("eval users:", len(eval_users))


eval users: 1600


## 5) Three retrievers (exact top-K)

In [6]:
S_beh = (U_beh[eval_users] @ I_beh.T).astype(np.float32)

B = rng.normal(size=(d_true, d_content)).astype(np.float32)
user_content_q = (U_true[eval_users] @ B + 0.6 * rng.normal(size=(len(eval_users), d_content)).astype(np.float32)).astype(np.float32)
user_content_q = l2_normalize(user_content_q)

S_cont = (user_content_q @ item_emb_content.T).astype(np.float32)

pop_rank = np.argsort(-pop)


## 6) Candidate fusion: quotas + dedupe

In [8]:
def topk_idx(scores_row, k):
    idx = np.argpartition(-scores_row, kth=k-1)[:k]
    return idx[np.argsort(-scores_row[idx])]

def fuse_candidates(i, K_final=500, q_beh=300, q_cont=150, q_pop=50):
    beh = topk_idx(S_beh[i], q_beh)
    cont = topk_idx(S_cont[i], q_cont)
    popc = pop_rank[:q_pop]

    seen = set()
    fused = []

    for arr in (beh, cont, popc):
        for x in arr:
            x = int(x)
            if x not in seen:
                fused.append(x); seen.add(x)
                if len(fused) >= K_final:
                    return np.array(fused, dtype=np.int32)

    # The original code continued to add from 'beh' if K_final was not reached.
    # This part can be simplified by just making sure we pad if not enough items are found.
    # Fill the remaining slots with a placeholder if fewer than K_final unique candidates were found.
    while len(fused) < K_final:
        fused.append(-1) # Using -1 as a placeholder for missing candidates

    return np.array(fused, dtype=np.int32)

K_final = 500
cands = np.stack([fuse_candidates(i, K_final=K_final) for i in range(len(eval_users))], axis=0)

# When calculating recall, make sure to exclude the placeholder value if it's not a valid item ID
rec = np.mean([1.0 if int(pos_targets[i]) in set(cands[i].tolist()) and int(pos_targets[i]) != -1 else 0.0 for i in range(len(eval_users))])
print(f"Candidate Recall@{K_final}:", round(rec, 4))

Candidate Recall@500: 0.1431


## 7) Compare retrievers

In [9]:
def recall_from_scores(S, k):
    return np.mean([recall_at_k(S[i], pos_targets[i], k=k) for i in range(len(eval_users))])

for k in [50, 200, 500]:
    r_beh = recall_from_scores(S_beh, k)
    r_cont = recall_from_scores(S_cont, k)
    r_fuse = np.mean([1.0 if int(pos_targets[i]) in set(cands[i,:k].tolist()) else 0.0 for i in range(len(eval_users))])
    print(f"Recall@{k}: behavior={r_beh:.4f}  content={r_cont:.4f}  fused≈{r_fuse:.4f}")


Recall@50: behavior=0.0088  content=0.0181  fused≈0.0088
Recall@200: behavior=0.0587  content=0.0550  fused≈0.0587
Recall@500: behavior=0.1494  content=0.1525  fused≈0.1431


## Production notes
- Measure retrieval coverage (candidate recall) before tuning ranking.
- Multi-retriever fusion improves robustness. We use behavior for personalization, content for cold-start, popularity for fallback.
- Quotas + dedupe prevent one retriever from dominating.
- Vectors, ANN index, and code move together as one versioned unit.
